# NO skip user train

In [1]:
import random
import pickle
from pathlib import Path

import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from tqdm.notebook import tqdm, trange

from conv_nets.wesad_tries.models.net_up_down_coder3_30 import (
    NetUpDownCoder3_30 as Model,
)
from ds.wesad.datasets import subjects_data
from ds.wesad.datasets_users import SubjectsDataset

In [2]:
seed = 1337

random.seed(seed)
np.random.seed(seed)
torch.random.manual_seed(seed)

In [3]:
base_path = Path("/home/dmo/Documents/human_func_state/human_func_state")
base_path = base_path.joinpath(
    "models_dumps",
    "wesad",
    "subjects_related",
    "normalised",
    "common",
    "before_personalised",
)

In [4]:
epoch_count = 50
optim_lr = 1e-6
betas = "default"  # (0.85, 0.995) # Adam only
each_user_rate_history = True
ds_step_size = 5
train_batch_size = 8
test_batch_size = 1
device = "cuda"
Optimizer = optim.ASGD

In [5]:
net_with_optim_name = f"{Model.__name__}_{Optimizer.__name__}_lr_{optim_lr}"
base_path = base_path / net_with_optim_name

In [6]:
key = "rr_intervals_subject_normalised"
ds_train = SubjectsDataset(
    subjects_data,
    ds_type="train",
    step=ds_step_size,
    key=key,
)
ds_test = SubjectsDataset(
    subjects_data,
    ds_type="test",
    step=ds_step_size,
    key=key,
)
dl_train = DataLoader(
    ds_train,
    batch_size=train_batch_size,
    shuffle=True,
    num_workers=1,
    pin_memory=False,
    drop_last=True,
)
dl_test = DataLoader(
    ds_test,
    batch_size=test_batch_size,
    shuffle=True,
    num_workers=1,
    pin_memory=False,
    drop_last=True,
)

In [7]:
def get_net_criterion_optimizer() -> (
    tuple[Model, nn.CrossEntropyLoss, optim.Optimizer]
):
    net = Model().to(device=device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.ASGD(net.parameters())
    return net, criterion, optimizer

In [8]:
writer = SummaryWriter(log_dir=base_path / "hist")
dump_name = (base_path / f"{net_with_optim_name}_last").with_suffix(".pkl")
best_name = (base_path / f"{net_with_optim_name}_best").with_suffix(".pkl")

In [9]:
def val(model: Model, dl):
    with torch.no_grad():
        sum_ = 0
        for i, data in enumerate(dl):
            inputs, labels = data
            inputs = (
                torch.unsqueeze(inputs, 1).to(torch.float32).to(device=device)
            )

            outputs = model(inputs).cpu()
            eq = outputs.max(1).indices == labels
            sum_ += eq.sum()
    return sum_

In [10]:
def save_data(path, net_state_dict, optimizer, epoch, rate, rate_subject=None):
    with open(path, "wb") as f:
        pickle.dump(
            {
                "net_state_dict": net_state_dict,
                "current_epoch": epoch,
                "rate": rate,
                "rate_subject": rate_subject,
                "optimizer": optimizer.__class__.__name__,
                "optimizer_params": optimizer.param_groups,
                "train_batch_size": train_batch_size,
                "test_batch_size": test_batch_size,
                "device": device,
            },
            f,
        )

In [11]:
def train_epoch(
    model,
    criterion,
    optimizer,
    dl,
    epoch,
    min_loss,
    out_f,
    print_step=50,
):
    epoch_loss = 0.0
    running_loss = 0.0
    for i, data in enumerate(dl):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs = torch.unsqueeze(inputs, 1).to(torch.float32).to(device=device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs.cpu(), labels)
        loss.backward()
        optimizer.step()
        # print statistics
        running_loss += loss.item()
        if i % print_step == print_step - 1:
            mean_loss = running_loss / print_step
            if mean_loss < min_loss[0]:
                min_loss = (mean_loss, (epoch, i))
            print(f"[{epoch:3d}, {i:4d}] loss: {mean_loss:.3f}", file=out_f)
            epoch_loss += running_loss
            running_loss = 0.0
    epoch_loss += running_loss
    return epoch_loss

In [12]:
def train(
    out_f,
    epoch_count=50,
    print_step=50,
    start_epoch=0,
):
    min_loss = (torch.tensor(np.inf), 0)
    best_common_rate = (torch.tensor(0.0), 0)
    worst_common_rate = (torch.tensor(1.0), 0)
    best_single_rate = (torch.tensor(0.0), 0)
    worst_single_rate = (torch.tensor(1.0), 0)
    model, criterion, optimizer = get_net_criterion_optimizer()
    # loop over the dataset multiple times
    for epoch in trange(start_epoch, epoch_count):
        model.train()
        epoch_loss = train_epoch(
            model,
            criterion,
            optimizer,
            dl_train,
            epoch,
            min_loss,
            out_f,
            print_step,
        )
        model.eval()
        common_acc = val(model, dl_test)
        writer.add_scalar("Loss/train", epoch_loss, epoch)
        writer.add_scalar("Accuracy/train", common_acc, epoch)
        if common_acc > best_common_rate[0]:
            best_common_rate = (common_acc, epoch)
            save_data(
                best_name, model.state_dict(), optimizer, epoch, common_acc
            )
        if common_acc < worst_common_rate[0]:
            worst_common_rate = (common_acc, epoch)
        save_data(dump_name, model.state_dict(), optimizer, epoch, common_acc)

        print(
            (
                f"[{epoch:3d}] rate: {common_acc:.4f}; "
                f"{best_common_rate = }, {worst_common_rate = }"
            ),
            file=out_f,
        )
    return (
        worst_common_rate,
        best_common_rate,
        min_loss,
        worst_single_rate,
        best_single_rate,
    )

In [13]:
with open(base_path / "train.log", "w") as out_f:
    train(out_f, epoch_count=epoch_count)

  0%|          | 0/50 [00:00<?, ?it/s]